# Titkosított üzenetek

Megtudtátok a barátaitokkal, hogy lehallgatják a beszélgetéseiteket. Ezért kitaláltátok, hogy saját kódolt morse üzenetekkel fogtok kommunikálni, aminek az értelmezését (dekódolását) csak ti ismeritek. 

Ki kell találni, hogy a szavakat hogyan kódolnátok hosszú és rövid idejű gomblenyomásokkal. Ha ez meg van és elkészült az üzenet is, akkor azt le kell menteni egy txt fájl formájában, elküldeni azt a barátodnak, akik a dekódolót lefuttatva a fájlon már az általad megfogalmazott szöveget látja. 


## Mit fogsz készíteni?

Ebben a gyakorlatban egy gombot fogunk használni arra, hogy kódolt üzeneteket hozzunk létre és azok értelmezéséhez írunk egy dekódolót is. Mindehhez néhány dolgot viszont érdemes előre tisztázni:

* Ha a gombot kevesebb mint 1 s-ig nyomjuk le, azt rövid jelnek tekintjük és 's'-sel (short) jelöljük.
* Ha a gombot 1-3 s-ig tartjuk lenyomva akkor, azt hosszú jelnek tekintjük és 'l'-lel (long) jelöljük.
* Hogy a különböző szavakat meg tudjuk különböztetni, azokat új sorba mentjük el, és új sort, '\n', akkor tudunk létrehozni ha a gombot 3-7 másodpercig tartjuk nyomva.
* Ha a gombot 7 másodpercnél tovább tartjuk lenyomva, akkor jelezzük a kódnak, hogy vége az üzenetnek. 

A dekódoló programnak tartalmaznia kell egy leképezést, ami a hosszú és rövid jelek kombinációját szavakká alakítja. Mindezt egy ```dictionary``` típusú változóval oldjuk meg. 

## Mit tanulsz meg?

* Hogyan kell fájlokat megnyitni, módosítani és bezárni pythonban
* Hogyan programozd be a Raspberry Pi GPIO kimeneteleit a **gpiozero** modullal
* Hogyan használjunk a gombot időtartam mérésre
* Hogyan hozhatunk létre függvényeket
* Hogyan használunk *if-else* szerkezeteket pythonban
* Hogyan dolgozunk *dictionary* típusokkal
* Hogyan használunk *for* ciklusokat

## A projekt részletekre bontása

* Mérni kell a gomblenyomások hosszát
* *if-else* szerkezettel eldönteni, hogy a mért gombnyomás rövid, hosszú, újsor vagy üzenet vége jel
* fájlba menteni az üzenetet
* a dekódolóban beolvasni a fájl tartalmát
* dekódolni az üzenetet

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) Nyomó gomb: [itt vásárolhatsz](https://hu.farnell.com/schurter/1301-9320/switch-smd-push-12-5mm/dp/1217772?gclid=Cj0KCQjwzZj2BRDVARIsABs3l9K-ACTnuRr-dLVcDUKleNfECM3H3kWS_RfWtmXGMXVJeY9otda4dDcaAvGLEALw_wcB&gross_price=true&mckv=sSFnjERxR_dc|pcrid|434487710397|plid||kword||match||slid||product|1217772|pgrid|101346804139|ptaid|pla-389347076066|&CMP=KNC-GHU-SHOPPING-switches-relays-NEWSTRUCTURE-13-MAY-20) vagy [építhetsz]()

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

## A kapcsolási rajz

<img src="prog02_circuit.png" width=600 height=400 />

1) A nyomógombot lábait illesszük a breadbord mélyedéseibe. Ügyeljünk arra, hogy a nyomógomb egy (vízszintes) sorba kerülő lábai a gomb lenyomása nélkül is kapcsolatban legyenek egymással, míg a (függőlegesen) egy sorba kerülő lábai csak a lenyomás következtében lesznek összekötve. Mindezt egy műszerrel leellenprízhetjük.

2) Az egyik jumper kábelt (kék) kössük a gomb egyik lábának sorába, míg a másik végét a Raspberry Pi **GPIO 02**-es (vagy **3**-as) jelöléssel ellátott tüskéjére. Ez a tüske alapértelmezetten pull-up (magas vagy 1) módban van, azaz 3.3 V-ot mutat a földeléshez képest.

3) A másik jumper kábelt (narancssárga) kössük a gomb másik lábának sorába, míg a másik végét a Raspberry Pi **GND** (pl. **6**-os) jelöléssel ellátott tüskéjére. Ez a tüske alapértelmezetten földelés.

## A kód

Miután elkészítettük az áramkört, meg kell írnunk a kódot ami utasítja a Raspberry Pi-t, hogyan viselkedjen attól függően, hogy meddig tartották lenyomva a gombot.

Először betöltjük a ```gpiozero``` modulból a ```Button``` objektumot ami lehetővé teszi, hogy a Raspi kommunikáljon a gombbal a **GPIO** tüskéken keresztül. Emellett betöltjük a **time** modult amit időmérésre fogunk majd használni.

In [ ]:
from gpiozero import Button
import time

Az objektum beolvasása után inicializálunk egy ```Button``` objektumot, amit ```button```-nak nevezünk el. Egyben megmondjuk az objektumnak, hogy a fizikai gombot a *2*-es számú **GPIO** tüskére csatlakoztattuk. Emellett két változót is inicializálunk, ahol az első, ```press``` a lenyomás pillanatát méri majd, a második ```release``` pedig a gomb felengedésének pillanatát rögzíti.

In [ ]:
button = Button(2)
press = 0
release = 0

Majd az ```open``` paranccsal megnyitunk egy szöveges (text) fájlt, amit ```message.txt``` névre keresztelünk. Az ```open``` első argumentuma a fájl neve kell legyen (tartalmazva az avszolút vagy relatív mappa pozíciót is). Ebbe a fájlba fogjuk rögzíteni a titkos kódolt üzenetet, amit később majd egy dekódoló olvas be és értelmez. 

In [ ]:
msg = open('message.txt','w')

Ezután definiáljuk azokat a függvényeket, amik a gomb lenyomva tartásának idejét segítenek mérni és egyben elvégzik az üzenez kódolását is. Az első a ```start``` függvény, amit a gomb lenyomása indít majd el. Ez a függvény a ```press``` globális változóba menti ell a gomb lenyomásának pillanatát.

In [ ]:
def start():
    global press
    press = time.time()

A második függvény, ```end```, feladata három rétű (ld. a kommentelt részt a függvény belsejében: Part 1, 2 3). Első körben a ```release``` globális változóba menti a gomb felengedésének pillanatát a ```time.time()``` időmérővel. Ezután kiszámolja, mennyi ideig volt nyomva tartva a gomb és azt a ```delay``` változóba tárolja. Ugyanitt még beolvassa a ```msg``` globális változót, ami a szöveges fájl kezeléséért felelős, amibe a kódolt üzenetet mentjük majd. 

In [ ]:
def end():
    ### Part 1 ###
    global release, msg
    release = time.time()
    delay = release - press
    ### Part 2 ###
    code = encoder(delay)
    ### Part 3 ###
    if code == 'close':
        msg.close()
        print('Kódolt üzenet elmentve')
        print('Kérem lépjen ki a programból!')
    if not msg.closed:
        msg.write(code)

A második részben (Part 2) a ```delay``` értéke alapján megpróbáljuk eldönteni, milyen kódolt üzenetet kell átadni. Erre az ```encoder(delay)``` függvényt használjuk, ami bemenetként a lenyomva tartás idejét várja és kimenetként 's' (short), 'l' (long), '\n' (új sor) vagy 'close' értéket ad vissza és tárol a ```code``` változóban. Ezt a függvényt lejjebb részletezzük.

A harmadik rész pedig két ```if``` feltételes szerkezettel dönti el mit csináljon. Az első szerkezet megvizsgálja, hogy a ```code``` értéke megegyezik-e a ```'close'``` stringgel. Ha igen, akkor utasítást ad a szöveges fájl bezárására és annak tartalmának mentésére  ```msg.close()``` valamint kiírja a képernyőre, hogy az üzenet elmentve és Kérem lépjen ki a programból.

A ```code``` értékének megvizsgálása után következik még egy ```if``` szerkezet. Ez azt vizsgálja meg, hogy a szöveges fájl nyitva van-e még ```msg.closed```. Az ```if``` után a ```not``` negálja az utána következő kifejezést, így pl. ha aszöveges fájl nincs zárva van, akkor az ```msg.closed``` értéke ```False```, amit ha negálunk a ```not``` kifejezéssel, akkor ```True```-t kapunk és végrehajtódik az ```if``` szerkezet. Itt csak arra utasítjuk a gépet, hogy írja a szöveges fájlba a kódolt üzenetet: ```msg.write(code)```.   

Most nézzük meg mit csinál az ```encoder(delay)``` függvény. Ez egy többszörös ```if-elif-else``` szerkezetet tartalmaz. Először megnéz, hogy a gomb lenyomásának hossza beleesik-e az 1-3 másodperc intervallumba. Ha igen, akkor ez egy hosszú jel, és visszaküldi a 'l' stringet. Ha nem esik bele, akkor megnézi, hogy rövidebb-e 1 másodpercnél. Ha igen, akkor ez egy rövid jel, és visszaküldi a 's' stringet. Ha a lenyomás ideje a 3-7 másodperc intervallumba esik, akkor az egy új sort jelent, és visszaküld egy '\n' jelet. Végül, ha mindezen feltételek közül egyik sem teljesül, azaz a gomb 7 másodpercnél tovább volt lenyomva, akkor az azt jelenti, hogy be kell zárni a fájlt, vége az üzenetnek. Ebben az esetben a visszaküldött string a 'close'. 

In [ ]:
def encoder(delay):
    if delay >= 1 and delay < 3:
        return 'l'
    elif delay < 1:
        return 's'
    elif delay >= 3 and delay < 7:
        return '\n'
    else:
        return 'close'

Az itt definiát döntéshozatali szabályok teljesen önkényesek és tetszés szerint változtathatók. 

Ezután már csak hozzá kell rendelnünk a gomb lenyomásához és felengedéséhez a megfelelő fügvvényeket:

In [ ]:
button.when_pressed = start
button.when_released = end

Végül pedig elindítunk egy végtelen ciklust ami várja az üzeneteket. A ```pass``` parancs csak annyit, jelent, hogy nem szeretnénk semmilyen itt definiált utasítást végrehajtani, így a while ciklus csak ismétlődik örökké. 

In [ ]:
while True:
    pass

Miután megtörtént az üzenet kódolása és elmentése az egyik fél részéről, a másik félnek tudnia kell azt dekódolni. Erre való az itt következő dekódoló program. 

Először definiálunk egy ```code_map``` nevű dictionary változót:

In [1]:
code_map = {'trambulin':'lsls',
            'szia':'l',
            'gyere':'ll',
            'mizu':'sl',
            'siess':'sss',
}

A dictionary az a python egyik alap típusa, lényegében rendezetlen kulcs-érték párokat tartalmaz. Egy értékre a változó nevével és az azt követő megfelelő kulccsal lehet hivatkozni szögletes zárójelben. Pl. a 'szia' kulcshoz tartozó érték:

In [2]:
code_map['szia']

'l'

Van néhány hasznos metódusa a dictionary-knek, amiket a következőkben fel is fogunk használni:

Listába rendezni a kulcsokat, listába rendezni az értékeket és listába rendezni a kulcs-érték párokat (fontos, hogy a kapott eredmény egyik esetben sincs sorrendbe rakva):

In [3]:
code_map.keys()

dict_keys(['trambulin', 'szia', 'gyere', 'mizu', 'siess'])

In [4]:
code_map.values()

dict_values(['lsls', 'l', 'll', 'sl', 'sss'])

In [5]:
code_map.items()

dict_items([('trambulin', 'lsls'), ('szia', 'l'), ('gyere', 'll'), ('mizu', 'sl'), ('siess', 'sss')])

A kapcsos zárójelben az átlagos reakció idő változójának nevére hivatkozunk, hogy kinyomtassuk és a ```:.2f``` arra utal, hogy csak két tizedes pontosságig nyomtassuk ki. 

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```button_reflex.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```button_reflex.py```-t elmentettük. Ott begépelve a ```python button_reflex.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a gomb lenyomására majd elengedésére a képernyőn megjelenik a nyomvatartás hossza másodpercekben.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Kinyomtatni statisztikát a teljesítményről: átlag, standard deviáció, minimum és maximum érték.
* Elmenteni fájlba a játékos eredményét és összehasonlítani a következő játékos eredményével, így ranglistát készíteni.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) gpiozero leírás - [https://gpiozero.readthedocs.io/en/stable/recipes.html](https://gpiozero.readthedocs.io/en/stable/recipes.html)

2) time.time leírása - [https://docs.python.org/3/library/time.html](https://docs.python.org/3/library/time.html)

3) python f'' string - [https://realpython.com/python-f-strings/](https://realpython.com/python-f-strings/)

4) random modul leírás - [https://docs.python.org/3/library/random.html](https://docs.python.org/3/library/random.html)

5) os modul leírás - [https://docs.python.org/3/library/os.html](https://docs.python.org/3/library/os.html)

6) for ciklus leírás - [https://wiki.python.org/moin/ForLoop](https://wiki.python.org/moin/ForLoop)

7) range függvény - [https://snakify.org/en/lessons/for_loop_range/](https://snakify.org/en/lessons/for_loop_range/)